In [2]:
import pandas as pd

In [16]:
df = pd.read_excel('./naver_shopping_result.xlsx', header=None)

In [19]:
df.to_csv('./naver_shopping_result.csv', index=False, header=False, encoding='utf-8-sig')

In [18]:
df

,0,1
0,1,닭가슴살
1,2,오메가3
2,3,먹태깡
3,4,글루타치온
4,5,쌀20kg
...,...,...
495,496,마카
496,497,스테비아
497,498,하누카세
498,499,능이버섯


## pymysql 파이썬 라이브러리를 이용하여 mysql DB 연결 및 종료 코드를 작성

In [20]:
import sys
import pymysql

def mysqlDbConnection(u, pw, h, p, d):
    try:
        conn = pymysql.connect(user = u, password = pw, host = h, port = p, database = d, charset='utf8')
        print(f"DB Connection Success: {h}")
    except pymysql.Error as e:
        print(f"Error connecting to MySQL Platform : {e}")
        sys.exit(1)
    return conn


def mysqlDbClose(_dbConn):
    try:
        _dbConn.close()
        print("DB Close Success")
    except pymysql.Error as e:
        print("Error closing from MySQL Platform")
        sys.exit(1)


## mysql 스키마 및 테이블을 생성

In [26]:
import pymysql

# dbConn = pymysql.connect(host='localhost', user='rubi', password='mysql', db='shop100', charset='utf8')
dbConn = mysqlDbConnection('rubi',  'mysql', 'localhost', 3306, 'shop100')
cursor = dbConn.cursor()

# 테이블 생성(데이터 타입 : 공식 레퍼런스 참조 또는 검색 참조)
cursor.execute("CREATE TABLE IF NOT EXISTS example_csv (`ranking` bigint(20) NOT NULL, "
               "`item` varchar(50) NOT NULL, PRIMARY KEY (`ranking`))"
)

dbConn.commit()
cursor.close()
mysqlDbClose(dbConn)


DB Connection Success: localhost
DB Close Success


## csv 파이썬 라이브러리를 이용하여 파일 데이터를 추출하고, mysql DB에 저장하는 코드를 작성

In [31]:
import csv

dbConn = mysqlDbConnection('rubi',  'mysql', 'localhost', 3306, 'shop100')
cursor = dbConn.cursor()

file = open('./naver_shopping_result.csv','r', encoding='UTF8')
fReader = csv.reader(file)

# 첫 번째 라인을 읽어서 BOM 제거
first_line = next(fReader)
first_line = [cell.strip('\ufeff') for cell in first_line]

for line in fReader:
#     print(line[0], line[1])
    ranking = line[0]
    item = line[1]
    
    # 해당 rank 값이 테이블에 이미 있는지 확인
    query_check = "SELECT COUNT(*) FROM example_csv WHERE ranking = %s"
    cursor.execute(query_check, (ranking,))
    count = cursor.fetchone()[0]
    
    # 중복되지 않은 경우에만 INSERT 문 실행
    if count == 0:
        query_insert = "INSERT INTO example_csv VALUES (%s, %s)"
        cursor.execute(query_insert, (ranking, item))
    
file.close()

dbConn.commit() 
cursor.close()
mysqlDbClose(dbConn)

DB Connection Success: localhost
DB Close Success
